# Spatial History
Intro to Digital History, 4/4/2022
Portions adapted from https://melaniewalsh.github.io/Intro-Cultural-Analytics/07-Mapping/01-Mapping.html

In [1]:
#!pip install geopy
#!pip install folium
#!pip install geopandas

In [2]:
from geopy.geocoders import Nominatim
import folium

geolocator = Nominatim(user_agent="YOUR NAME's mapping app", timeout=2)

In [3]:
loc = "University of Utah"

location = geolocator.geocode(loc)

location

Location(University of Utah, 201, Presidents Circle, Salt Lake City, Salt Lake County, Utah, 84112, United States, (40.762813699999995, -111.83687191368261, 0.0))

In [4]:
print(location.address)

University of Utah, 201, Presidents Circle, Salt Lake City, Salt Lake County, Utah, 84112, United States


In [5]:
print(location.latitude, location.longitude)

40.762813699999995 -111.83687191368261


In [6]:
print(f"Class: {location.raw['class']} \nType: {location.raw['type']}")

Class: amenity 
Type: university


In [7]:
utah_map = folium.Map(location=[location.latitude, location.longitude],
                     zoom_start=14)

folium.Marker(location=[location.latitude, location.longitude],
             popup="University of Utah").add_to(utah_map)

utah_map

In [8]:
utah_map = folium.Map(location=[location.latitude, location.longitude],
                     zoom_start=10)

folium.CircleMarker(location=[location.latitude, location.longitude],
                    radius=10,
                    color = 'black',
                    fill_color = 'red',
                    fill_opacity = 0.6,
                    popup = "University of Utah").add_to(utah_map)

utah_map

In [9]:
possible_locations = geolocator.geocode("University of Utah", exactly_one=False)

for location in possible_locations:
    print(location.address)
    print(location.latitude, location.longitude)
    print(f"Importance: {location.raw['importance']}")

University of Utah, 201, Presidents Circle, Salt Lake City, Salt Lake County, Utah, 84112, United States
40.762813699999995 -111.83687191368261
Importance: 0.8557694184669417
Department of Linguistics, University of Utah, 255, Central Campus Drive, Salt Lake City, Salt Lake County, Utah, 84112, United States
40.7634795 -111.8408017
Importance: 0.31100000000000005
Outdoor Adventures, Presidents Circle, Salt Lake City, Salt Lake County, Utah, 84112, United States
40.7655129 -111.838165
Importance: 0.11100000000000002


# Where is Utah History?

Using Named Entity Recognition

In [10]:
#!pip install -U spacy
#!python -m spacy download en_core_web_sm
import spacy
from spacy import displacy
from collections import Counter
import pandas as pd
pd.options.display.max_rows = 600
pd.options.display.max_colwidth = 400
import en_core_web_sm
nlp = en_core_web_sm.load()

In [11]:
import pandas as pd

#Load UHQ data into a pandas dataframe
df = pd.read_excel("UHQ_FULL_EDITED.xlsx")

In [12]:
#Look at the first article in our database
text = df.text.tolist()[0]

#Run Named Entity Recognition on the text
document = nlp(text)

displacy.render(document[:500], style="ent")

In [13]:
#Create a list of all location (GPE) entities in the text
location_list = []
for named_entity in document.ents:
    if named_entity.label_ == "GPE":
        location_list.append(named_entity.text.replace("’s", ""))

#Print our list
location_list

['Salt Lake City',
 'Salt Lake City',
 'Salt Lake City',
 'Utah',
 'Salt Lake City',
 'Amanda',
 'Amanda',
 'California',
 'Salt Lake City',
 'Park City',
 'New York',
 'Utah',
 'Ogden',
 'Washington',
 'Utah',
 'Utah',
 'Utah',
 'Michigan',
 'Salt Lake City',
 'Salt Lake City',
 'Utah',
 'Utah',
 'New York',
 'Chicago',
 'Chicago',
 'Herald',
 'Russia']

### Question: What are some potential problems that you see with this list? 

In [14]:
#Is Amanda a place???
loc = "Amanda"

location = geolocator.geocode(loc)
location

Location(Amanda, Fairfield County, Ohio, United States, (39.649508, -82.7443367, 0.0))

In [15]:
#We can use the geolocator library to locate the latitude and longitude for each location. 

location_tally = Counter(location_list)

location_df = pd.DataFrame(location_tally.most_common(), columns=['location', 'count'])

loc_list = location_df.location.tolist()

lat = []
lon = []

for l in loc_list:
    temp_loc = geolocator.geocode(l)
    lat.append(temp_loc.latitude)
    lon.append(temp_loc.longitude)

location_df['lat'] = lat
location_df['lon'] = lon
location_df

,location,count,lat,lon
0,Salt Lake City,7,40.759620,-111.886798
1,Utah,7,39.422519,-111.714358
2,Amanda,2,39.649508,-82.744337
3,New York,2,40.712728,-74.006015
4,Chicago,2,41.875562,-87.624421
5,California,1,36.701463,-118.755997
6,Park City,1,40.646092,-111.497996
7,Ogden,1,41.223005,-111.973843
8,Washington,1,38.895037,-77.036543
9,Michigan,1,43.621195,-84.682435


Note that this returns a single point in space. Are Utah, California, or even Russia one point on a map? 

In [16]:
world_map = folium.Map(location=[42, -102], zoom_start=1)
location_df = location_df.dropna(subset=['lat', 'lon'])

lat = list(location_df["lat"])
long = list(location_df["lon"])
rad = list(location_df["count"])
name = list(location_df["location"])

for lt,ln,r,n in zip(lat,long,rad,name):
    folium.CircleMarker(location = [lt,ln],
                        popup=n,
                        radius = r).add_to(world_map)
world_map

## Using US Geological Survey Data

In [35]:
usgs_df = pd.read_csv("UTAH_USGS.csv")
len(usgs_df)

31913

In [36]:
usgs_df[usgs_df.STATE_ALPHA == "UT"].FEATURE_CLASS.value_counts()

Valley             5730
Populated Place    3618
Spring             2554
Stream             2545
Summit             2351
Locale             1926
Church             1577
Reservoir           961
Building            906
Lake                899
School              870
Flat                775
Mine                704
Canal               576
Ridge               554
Cemetery            531
Park                455
Basin               445
Civil               403
Dam                 384
Gap                 310
Post Office         259
Bench               243
Airport             221
Arch                200
Trail               188
Pillar              179
Cliff               177
Well                131
Range               113
Tower               112
Bend                106
Census               98
Area                 96
Hospital             88
Bay                  63
Rapids               56
Oilfield             52
Slope                45
Plain                44
Bar                  38
Bridge          

In [19]:
feature = "Populated Place"
ut_usgs_df = usgs_df[(usgs_df.STATE_ALPHA == "UT") & (usgs_df.FEATURE_CLASS == feature)]

In [20]:
ut_usgs_df[ut_usgs_df.FEATURE_NAME == "Salt Lake City"]

,FEATURE_ID,FEATURE_NAME,FEATURE_CLASS,STATE_ALPHA,STATE_NUMERIC,COUNTY_NAME,COUNTY_NUMERIC,PRIMARY_LAT_DMS,PRIM_LONG_DMS,PRIM_LAT_DEC,PRIM_LONG_DEC,SOURCE_LAT_DMS,SOURCE_LONG_DMS,SOURCE_LAT_DEC,SOURCE_LONG_DEC,ELEV_IN_M,ELEV_IN_FT,MAP_NAME,DATE_CREATED,DATE_EDITED
1342331,1454997,Salt Lake City,Populated Place,UT,49,Salt Lake,35.0,404539N,1115328W,40.760779,-111.891047,NaN,NaN,NaN,NaN,1300.0,4265.0,Salt Lake City North,12/31/1979,05/11/2011


In [21]:
year_begin = 1928 #UHQ first year is 1928
year_end = 2021 #UHQ end year is 2021

#creating string containing all UHQ texts
df2 = df[(df['text'].notna()) & (df['year'] >= year_begin) & (df['year'] <= year_end)].reset_index()

full_uhq_texts = ' '.join([str(elem) for elem in df2.text.tolist()])

In [22]:
usgs_counts = []

for fn in ut_usgs_df.FEATURE_NAME.tolist():
    usgs_counts.append(full_uhq_texts.count(fn))
    
ut_usgs_df['FEATURE_COUNTS'] = usgs_counts

<ipython-input-22-7b0bfdab6c26>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ut_usgs_df['FEATURE_COUNTS'] = usgs_counts


In [23]:
len(usgs_counts), len(ut_usgs_df)

(3618, 3618)

In [24]:
ut_usgs_df

,FEATURE_ID,FEATURE_NAME,FEATURE_CLASS,STATE_ALPHA,STATE_NUMERIC,COUNTY_NAME,COUNTY_NUMERIC,PRIMARY_LAT_DMS,PRIM_LONG_DMS,PRIM_LAT_DEC,...,SOURCE_LAT_DMS,SOURCE_LONG_DMS,SOURCE_LAT_DEC,SOURCE_LONG_DEC,ELEV_IN_M,ELEV_IN_FT,MAP_NAME,DATE_CREATED,DATE_EDITED,FEATURE_COUNTS
347050,395885,Utida,Populated Place,UT,49,Cache,5.0,415957N,1115831W,41.999092,...,NaN,NaN,NaN,NaN,1411.0,4629.0,Trenton,12/31/1979,NaN,2
1313673,1425039,Abraham,Populated Place,UT,49,Millard,27.0,392348N,1124305W,39.396620,...,NaN,NaN,NaN,NaN,1400.0,4593.0,Sutherland,12/31/1979,NaN,335
1313684,1425050,Adamsville,Populated Place,UT,49,Beaver,1.0,381530N,1124738W,38.258303,...,NaN,NaN,NaN,NaN,1685.0,5528.0,Adamsville,12/31/1979,NaN,1
1313730,1425096,Altamont,Populated Place,UT,49,Duchesne,13.0,402134N,1101711W,40.359398,...,NaN,NaN,NaN,NaN,1947.0,6388.0,Altamont,12/31/1979,03/21/2008,0
1313731,1425097,Altonah,Populated Place,UT,49,Duchesne,13.0,402405N,1101738W,40.401341,...,NaN,NaN,NaN,NaN,2034.0,6673.0,Altonah,12/31/1979,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2199708,2708039,Covered Bridge Canyon,Populated Place,UT,49,Utah,49.0,400209N,1113312W,40.035829,...,NaN,NaN,NaN,NaN,1559.0,5115.0,Spanish Fork Peak,11/04/2011,11/08/2011,0
2260035,2783908,Interlaken,Populated Place,UT,49,Wasatch,51.0,403230N,1112823W,40.541664,...,NaN,NaN,NaN,NaN,1805.0,5922.0,Heber City,04/28/2016,NaN,0
2263734,2791542,Cedar Highlands,Populated Place,UT,49,Iron,21.0,373815N,1130243W,37.637628,...,NaN,NaN,NaN,NaN,2311.0,7582.0,Cedar City,05/17/2018,05/06/2021,0
2268987,2797924,Emigration Canyon,Populated Place,UT,49,Salt Lake,35.0,404612N,1114535W,40.769979,...,NaN,NaN,NaN,NaN,1639.0,5377.0,Fort Douglas,03/04/2019,NaN,65


Question: Do you notice any potential problems with the displayed data above? Note that "Abraham" appears in UHQ 335 times, but do you think all of these occurances refer to the location?

In [25]:
place_name = "Salt Lake City"
ut_usgs_df[ut_usgs_df.FEATURE_NAME == place_name]

,FEATURE_ID,FEATURE_NAME,FEATURE_CLASS,STATE_ALPHA,STATE_NUMERIC,COUNTY_NAME,COUNTY_NUMERIC,PRIMARY_LAT_DMS,PRIM_LONG_DMS,PRIM_LAT_DEC,...,SOURCE_LAT_DMS,SOURCE_LONG_DMS,SOURCE_LAT_DEC,SOURCE_LONG_DEC,ELEV_IN_M,ELEV_IN_FT,MAP_NAME,DATE_CREATED,DATE_EDITED,FEATURE_COUNTS
1342331,1454997,Salt Lake City,Populated Place,UT,49,Salt Lake,35.0,404539N,1115328W,40.760779,...,NaN,NaN,NaN,NaN,1300.0,4265.0,Salt Lake City North,12/31/1979,05/11/2011,6049


In [26]:
utah_map = folium.Map(location=[39, -111], zoom_start=5)

min_number = 1

sub_df = ut_usgs_df[ut_usgs_df.FEATURE_COUNTS >= min_number]

lat = list(sub_df["PRIM_LAT_DEC"])
long = list(sub_df["PRIM_LONG_DEC"])
rad = list(sub_df["FEATURE_COUNTS"])
name = list(sub_df["FEATURE_NAME"])

for lt,ln,r,n in zip(lat,long,rad,name):
    folium.CircleMarker(location = [lt,ln],
                        popup=n,
                        tiles='Stamen Terrain',
                        radius = 1).add_to(utah_map)
utah_map

Try adjusting the "min_number" variable to see how your results change. Note that this approach is great at showing all of the places in Utah that are referenced at least 'n' number of times, but not great at showing volume. How much more likely is Salt Lake City or a place in Salt Lake County to appear than other locations in Utah? For that, we can turn to choropleth maps. 

## Choropleth Maps

In [27]:
#Import Libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from folium.features import GeoJsonTooltip

#Read the geoJSON file using geopandas
geojson = gpd.read_file('georef-united-states-of-america-county.geojson')
geojson=geojson[geojson['ste_name'] == "Utah"]

ut_county_map = folium.Map(location=[39.5, -111], zoom_start=7)

folium.Choropleth(geo_data=geojson).add_to(ut_county_map)
ut_county_map

In [28]:
min_number = 1

sub_df = ut_usgs_df[ut_usgs_df.FEATURE_COUNTS >= min_number]

county_counts = pd.DataFrame(sub_df['COUNTY_NAME'].value_counts().reset_index())
county_counts.columns = ['county_name', 'loc_count']
total_count = []
for county in county_counts.county_name.tolist():
    #sub_df = ut_usgs_df[ut_usgs_df.COUNTY_NAME == county]
    total_count.append(sub_df[sub_df.COUNTY_NAME == county]['FEATURE_COUNTS'].sum())

county_counts['total_count'] = total_count

county_counts

,county_name,loc_count,total_count
0,Salt Lake,304,36060
1,Davis,145,20336
2,Weber,113,12296
3,Utah,63,6595
4,Box Elder,46,2626
5,Tooele,31,2948
6,Cache,31,2638
7,Millard,31,4759
8,Washington,28,4519
9,Carbon,25,2078


In [29]:
df_final = geojson.merge(county_counts, left_on="coty_name", right_on="county_name", how="outer") 
df_final = df_final[~df_final['geometry'].isna()]
df_final

,coty_code,coty_gnis_code,coty_name_long,ste_name,coty_type,year,coty_area_code,ste_code,coty_fp_code,coty_name,geometry,county_name,loc_count,total_count
0,49037,01448032,San Juan County,Utah,county,2020,USA,49,037,San Juan,"POLYGON ((-109.92799 38.15188, -109.92719 38.14796, -109.92937 38.13992, -109.93323 38.13630, -109.94366 38.13516, -109.95677 38.12945, -109.96269 38.11937, -109.96364 38.11433, -109.96950 38.11198, -109.97793 38.11010, -109.98214 38.10679, -109.98504 38.10178, -109.99250 38.09995, -109.99595 38.10052, -110.00102 38.10663, -110.00576 38.10955, -110.00949 38.10966, -110.01668 38.10613, -110.026...",San Juan,20,1328
1,49043,01448035,Summit County,Utah,county,2020,USA,49,043,Summit,"POLYGON ((-111.26497 41.14404, -111.27467 41.14257, -111.28972 41.14326, -111.29444 41.13913, -111.30009 41.13717, -111.30162 41.13442, -111.30701 41.13093, -111.31020 41.13049, -111.31846 41.13244, -111.32412 41.13745, -111.33239 41.13810, -111.33787 41.13304, -111.34096 41.13282, -111.34579 41.12872, -111.34886 41.12997, -111.36037 41.12919, -111.36660 41.13002, -111.36868 41.12201, -111.367...",Summit,22,2473
2,49041,01448034,Sevier County,Utah,county,2020,USA,49,041,Sevier,"POLYGON ((-112.51850 38.51041, -112.44870 38.51042, -112.38000 38.51050, -112.32676 38.51085, -112.28840 38.51135, -112.26964 38.51215, -112.12575 38.51214, -112.06213 38.51047, -112.06197 38.50209, -112.04655 38.50191, -112.00166 38.50194, -112.00075 38.50223, -111.96100 38.50218, -111.83230 38.50219, -111.76639 38.50224, -111.75630 38.51015, -111.62526 38.51022, -111.56074 38.51008, -111.443...",Sevier,21,3621
3,49051,01448039,Wasatch County,Utah,county,2020,USA,49,051,Wasatch,"POLYGON ((-110.89166 39.89965, -110.89197 40.00172, -110.89179 40.08240, -110.89627 40.08238, -110.89657 40.16383, -110.89666 40.21241, -110.89669 40.29772, -110.89675 40.36482, -110.89659 40.43118, -110.90293 40.43121, -110.90288 40.53457, -110.90248 40.58658, -110.90229 40.63465, -110.90183 40.64908, -110.90197 40.67816, -110.90195 40.68219, -110.91121 40.68199, -110.92472 40.68011, -110.933...",Wasatch,13,907
4,49021,01448025,Iron County,Utah,county,2020,USA,49,021,Iron,"POLYGON ((-114.04998 38.14876, -114.05004 38.09753, -114.05044 38.05425, -114.05029 38.03962, -114.05027 37.97947, -114.05050 37.95334, -114.05014 37.89481, -114.04966 37.88138, -114.04997 37.86785, -114.04947 37.83795, -114.04961 37.82354, -114.04830 37.80942, -114.04915 37.79491, -114.04986 37.76614, -114.05165 37.75150, -114.05201 37.67959, -114.05235 37.60478, -113.93803 37.60482, -113.863...",Iron,19,1419
5,49017,01448023,Garfield County,Utah,county,2020,USA,49,017,Garfield,"POLYGON ((-112.47868 38.14742, -112.47873 38.08691, -112.47866 37.99997, -112.47897 37.97768, -112.46697 37.97766, -112.46694 37.95692, -112.46818 37.89046, -112.50080 37.89040, -112.58840 37.89042, -112.58847 37.88221, -112.57840 37.88215, -112.57855 37.80454, -112.64339 37.80670, -112.67213 37.80553, -112.68922 37.80559, -112.68916 37.72624, -112.68899 37.71721, -112.68576 37.71734, -112.686...",Garfield,14,1593
6,49011,01448020,Davis County,Utah,county,2020,USA,49,011,Davis,"POLYGON ((-112.26022 40.76909, -112.14827 40.84613, -112.00657 40.92185, -111.95867 40.92182, -111.95860 40.90017, -111.96174 40.89336, -111.96615 40.89165, -111.96873 40.88690, -111.96756 40.88006, -111.97107 40.87664, -111.96289 40.87000, -111.96407 40.85997, -111.96588 40.85463, -111.95597 40.84921, -111.95162 40.84157, -111.94673 40.83796, -111.94287 40.83049, -111.94669 40.82384, -111.946...",Davis,145,20336
7,49009,01448019,Daggett County,Utah,county,2020,USA,49,009,Daggett,"POLYGON ((-109.04894 40.66260, -109.04909 40.71450, -109.04879 40.75143, -109.04846 40.82608, -109.04884 40.83782, -109.04888 40.88776, -109.04928 40.90865, -109.05003 41.00069, -109.17869 41.00090, -109.19781 41.00118, -109.21325 41.00201, -109.23251 41.00211, -109.25001 41.00105, -109.30735 41.00052, -109.40023 40.99988, -109.51089 40.99905, -109.54370 40.99840, -109.62095 40.99835, 

In [30]:
ut_county_map = folium.Map(location=[39.5, -111], zoom_start=7)

map_variable = 'total_count' #possible variables: 'loc_count' & 'total_count'

folium.Choropleth(
            geo_data=geojson,
            data=df_final,
            columns=['coty_code', map_variable],  #Here we tell folium to get the county fips and plot new_cases_7days metric for each county
            key_on='feature.properties.coty_code', #Here we grab the geometries/county boundaries from the geojson file using the key 'coty_code' which is the same as county fips
            #threshold_scale=(df_final[map_variable].quantile((0, 0.2, 0.4, 0.6, 0.8, 1))).tolist(), #we can create our own custom scale if we like.
            fill_color='YlOrRd',
            nan_fill_color="White", #Use white color if there is no data available for the county
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='', #title of the legend
            highlight=True,
            line_color='black').add_to(ut_county_map)

ut_county_map